### First Use Examples
The ETMClient is the link between the ETM API and Python. In order to interact with a scenario, the ETMClient first needs to connect to a scenario in the ETM. The scenario to which the client is connected is referenced to with a scenario_id. There are several ways to connect to a scenario, you can create a new scenario, you can create a copy of existing scenario's or you can connect to an existing scenario. First time users probably want to create an entire new scenario or a scenario that is based on a predefined template.

In [1]:
from pyETM import ETMClient

# initiate an empty client to establish a connection with the ETM
client = ETMClient(scenario_id=None, context='notebook')

# fetch the scenario templates from existing scenarios.
templates = client.get_scenario_templates()

# show first templates
templates.head()

,id,title,area_code,start_year,end_year,description,url,ordering,display_group,scaling
0,155680,100% duurzame energie in 2030. Het kan als je ...,nl,2015,2030,<span class='nl'>\n <p>Quintel Intelligence h...,https://engine.energytransitionmodel.com/api/v...,None,National scenarios,None
1,364762,West- en Midden-Brabant - regionale energiestr...,RGNB01_west_en_midden_brabant,2016,2050,<h1>West- en Midden-Brabant 2050<h1>\n<h2>Scen...,https://engine.energytransitionmodel.com/api/v...,None,Scenarios for provinces and regions,None
2,368636,West- en Midden-Brabant - variatie met wind op...,RGNB01_west_en_midden_brabant,2016,2050,<h1>West- en Midden-Brabant 2050<h1>\n<h2>Scen...,https://engine.energytransitionmodel.com/api/v...,None,Scenarios for provinces and regions,None
3,369649,Net voor de Toekomst - Regie Regionaal,nl,2015,2050,<span class='nl'><h2>Net voor de Toekomst - Sc...,https://engine.energytransitionmodel.com/api/v...,None,National scenarios,None
4,369650,Net voor de Toekomst - Regie Nationaal,nl,2015,2050,<h2>Net voor de Toekomst - Scenario 2 - Regie ...,https://engine.energytransitionmodel.com/api/v...,None,National scenarios,None


In [2]:
# select a template scenario
template_id = '36950'

# create a copy of the template scenario
client.create_scenario_copy(template_id)

# ask for the assigned scenario_id
scenario_id = client.scenario_id

# show id
scenario_id

'856103'

In [3]:
# you can reconnect to your scenario with the provided scenario_id
client = ETMClient(scenario_id=scenario_id, context='notebook')

# print scenario id
client.scenario_id

'856103'

### Scenario properties
After a connection with a scenario is established, you can request specific properties of that given scenario. The more basic properties that can be accessed for example are the title, start year, end year and area code of the scenario. 



In [4]:
# show title of the scenario
client.title

'API'

In [5]:
# set title of the scenario
client.title = 'Example scenario'

# show title of the scenario
client.title

'Example scenario'

More relevant properties of the scenario are the user configured parameters of the scenarios. You can access and change them via the user values property of the scenario.

In [6]:
# frst check which parameters can be set in the scenario
parameters = client.user_parameters

# show several parameters
parameters.iloc[41:46]

,min,max,default,share_group,disabled,user,cache_error,permitted_values
capacity_of_energy_power_nuclear_gen2_uranium_oxide,0.0,14790.873522,509.625,NaN,NaN,NaN,NaN,NaN
capacity_of_energy_chp_supercritical_waste_mix,0.0,23665.397635,726.2,NaN,NaN,NaN,NaN,NaN
capacity_of_energy_power_supercritical_waste_mix,0.0,28860.241018,0.0,NaN,NaN,NaN,NaN,NaN
capacity_of_energy_power_wind_turbine_coastal,0.0,6765.000000,950.411765,NaN,NaN,NaN,NaN,NaN
capacity_of_energy_power_wind_turbine_offshore,0.0,297534.000000,323.571429,NaN,NaN,NaN,NaN,NaN


In [7]:
# show parameters that are set by the user
user_values = client.user_values

# none have been set yet
user_values

Series([], Name: user, dtype: object)

In [8]:
# let's the capacities of several plants
# we can specify these changes in a dictonairy or series object.
user_values = {
    'capacity_of_energy_power_nuclear_gen2_uranium_oxide' : 750,
    'capacity_of_energy_power_wind_turbine_coastal' : 1500,
    'capacity_of_energy_power_wind_turbine_offshore' : 500
}

# apply the changes to the scenario
client.user_values = user_values

# check the results
client.user_values

capacity_of_energy_power_nuclear_gen2_uranium_oxide     750.0
capacity_of_energy_power_wind_turbine_coastal          1500.0
capacity_of_energy_power_wind_turbine_offshore          500.0
Name: user, dtype: object

As the user_values can be passed in different formats, it is possible to load a json as a dictonairy or a csv-file as series or dataframe with a 'user' column. When parameters are passed with a value outside the domain that is specified, the client will raise an error. 

### Result Curves
Simular to the scenario properties, the results of a scenario can also be accessed as properties of the scenario. When scenario parameters are changed, all results curves are automatically reset and are requested again upon accessing the client property. This means that the first time that a result curve is requested some time will pass before the result is loaded, as the ETM is evaluating the scenario in the background.

In [9]:
# request hourly electricity curves
ecurves = client.hourly_electricity_curves

ecurves.head()

,Time,buildings_solar_pv_solar_radiation.output (MW),energy_chp_combined_cycle_network_gas.output (MW),energy_chp_local_engine_biogas.output (MW),energy_chp_local_engine_network_gas.output (MW),energy_chp_local_wood_pellets.output (MW),energy_chp_supercritical_waste_mix.output (MW),energy_chp_ultra_supercritical_coal.output (MW),energy_chp_ultra_supercritical_cofiring_coal.output (MW),energy_chp_ultra_supercritical_lignite.output (MW),...,transport_bicycle_using_electricity.input (MW),transport_bus_using_electricity.input (MW),transport_car_flexibility_p2p_electricity.input (MW),transport_car_using_electricity.input (MW),transport_freight_train_using_electricity.input (MW),transport_motorcycle_using_electricity.input (MW),transport_passenger_train_using_electricity.input (MW),transport_tram_using_electricity.input (MW),transport_truck_using_electricity.input (MW),deficit
0,2050-01-01 00:00,0.0,187.2666,124.7358,0.0,29.0768,653.58,1216.9125,0.0,0.0,...,0.1633,0.6779,0.0,7.0363,32.0425,0.0134,47.8428,6.6211,0.5396,0.0
1,2050-01-01 01:00,0.0,187.2666,124.7358,0.0,29.0768,653.58,1216.9125,0.0,0.0,...,0.0608,0.5123,0.0,2.6199,24.2145,0.0050,17.8140,2.4653,0.4078,0.0
2,2050-01-01 02:00,0.0,187.2666,124.7358,0.0,29.0768,653.58,1216.9125,0.0,0.0,...,0.0286,0.4021,0.0,1.2336,19.0064,0.0024,8.3876,1.1608,0.3201,0.0
3,2050-01-01 03:00,0.0,187.2666,124.7358,0.0,0.0000,653.58,1216.9125,0.0,0.0,...,0.0199,0.3172,0.0,0.8575,14.9908,0.0016,5.8308,0.8069,0.2525,0.0
4,2050-01-01 04:00,0.0,187.2666,124.7358,0.0,0.0000,653.58,1216.9125,0.0,0.0,...,0.0199,0.2489,0.0,0.8579,11.7668,0.0016,5.8334,0.8073,0.1982,0.0


### Custom Curves
It is also possible to upload custom curves for a select number of parameters.

In [10]:
# show if there are custom curves attached in the scenario
client.ccurves.attached

key
interconnector_1_price             False
interconnector_2_price             False
interconnector_3_price             False
interconnector_4_price             False
interconnector_5_price             False
interconnector_6_price             False
weather/buildings_heating          False
electric_buses                     False
electric_trucks                    False
electric_vehicle_profile_1         False
electric_vehicle_profile_2         False
electric_vehicle_profile_3         False
electric_vehicle_profile_4         False
electric_vehicle_profile_5         False
industry_chemicals_electricity     False
industry_chemicals_heat            False
industry_ict                       False
industry_metals_electricity        False
industry_metals_heat               False
industry_other_electricity         False
industry_other_heat                False
industry_refineries_electricity    False
industry_refineries_heat           False
hydrogen_export                    False
hydrogen_imp

Set some scenarios by passing random data for the interconnectors

In [11]:
import numpy as np
import pandas as pd

# create prices for interconnectors
con1 = pd.Series(np.random.rand(8760)*3, name='interconnector_1_price')
con2 = pd.Series(np.random.rand(8760)*49, name='interconnector_2_price')
con3 = pd.Series(np.random.rand(8760)*5, name='interconnector_3_price')

# create capacity factors for wind
offshore = pd.Series(np.random.rand(8760), name='weather/wind_offshore_baseline')
onshore = pd.Series(np.random.rand(8760), name='weather/wind_inland_baseline')

# merge series in dataframe
ccurves = pd.concat([con1, con2, offshore, onshore], axis=1)

# show random data
ccurves.head()

,interconnector_1_price,interconnector_2_price,weather/wind_offshore_baseline,weather/wind_inland_baseline
0,2.232629,18.523040,0.397266,0.388992
1,1.630927,15.477090,0.289960,0.472343
2,2.921547,45.354484,0.502530,0.563405
3,0.364423,12.779884,0.816576,0.539170
4,1.507405,7.543705,0.512573,0.016567


In [12]:
# set data as ccurves profiles and specify dataset
client.upload_ccurves(ccurves, name='DataFrame upload')

# this also works for individual series
client.upload_ccurve(con3, name='Series upload')

# show uploaded ccurves
curves = client.ccurves.overview
curves = curves[curves.attached == True]

# show ccurves
curves

,type,display_group,overrides,attached,name,size,datetime,source_scenario
key,,,,,,,,
interconnector_1_price,price,interconnectors,0,True,DataFrame upload,42957.0,2021-08-06 15:07:44+00:00,{}
interconnector_2_price,price,interconnectors,0,True,DataFrame upload,49866.0,2021-08-06 15:07:45+00:00,{}
interconnector_3_price,price,interconnectors,0,True,Series upload,42899.0,2021-08-06 15:07:46+00:00,{}
weather/wind_offshore_baseline,capacity_profile,electricity_production,2,True,DataFrame upload,195847.0,2021-08-06 15:07:45+00:00,NaN
weather/wind_inland_baseline,capacity_profile,electricity_production,2,True,DataFrame upload,195861.0,2021-08-06 15:07:46+00:00,NaN
